# License

In [ ]:
# Copyright 2021 University of San Andres' Authors.

In [ ]:
#@title MIT License
#
# Copyright (c) 2022 University of San Andres
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Diseña tu propia red neuronal: clasificación de flores

![Tulip Field](https://hips.hearstapps.com/hbu.h-cdn.co/assets/17/09/1488567815-index-european-tulips.jpg)

El objetivo de este práctico es definir nuestra propia red neuronal, es decir, la arquitectura que ésta debe tener de tal modo de conseguir una clasificación de flores lo más robusta posible.

Los puntos a atacar son los siguientes:

* Descargar el set de datos de imágenes de flores (3700 fotos).
* Visualizar las imágenes y los distintos tipos de flores (labels).
* Separar el set de datos en 3 subconjuntos de datos: `train`, `val`, `test`.
* Preprocesar los datos, una parte muy importante en algoritmos que aprenden.
* Definir arquitectura de la red neuronal.
* Ajustar hiperparámetros, por ejemplo, *learning rate*, *epochs*, etc.
* Entrenar y verificar cuán bien entrenamos.
* Aumento de datos para un mejor entrenamiento.
* Verificación final de los resultados.



## Primero los datos

Ningún problema de Machine Learning (ML) puede ser atacado sin los datos, es por eso que en esta ocasión bajaremos imágenes de muchos tipos de flores de un set de datos públicos. Para esto utilizaremos *Python*, el lenguaje de programación más usado en aplicaciones de ML.

La siguiente celda importará código disponible por desarrolladores y científicos en el mundo para que podamos descargar los datos.

In [ ]:
%load_ext tensorboard

import PIL
import pathlib
import numpy as np
import tensorflow as tf

print(tf.__version__)

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print('Num GPU available:', len(gpu_devices))

Luego, procedemos a descargar los datos del sitio: https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

Con tan sólo ejecutar la siguiente celda, Python comenzará a descargar los datos por nosotros.

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

Un paso importante es saber cuántas imágenes tenemos disponibles para poder trabajar. Ejecutando siguiente celda de código nos dirá exactamente cuantas fotos tenemos.

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

### Visualización de los datos

Lo primero que vamos a hacer antes de comenzar a trabajar, es visualizar las fotos que tenemos. La siguientes celdas nos mostrarán una foto de cada tipo de flor que tenemos en el set de datos.

Las clases de flores son:
* Rosas (*Roses*).
* Tulipanes (*Tulips*).
* Margaritas (*Daisies*).
* Diente de León (*Dandelion*).
* Girasol (*Sunflower*).



In [ ]:
rose = str(list(data_dir.glob('roses/*'))[0])
tulip = str(list(data_dir.glob('tulips/*'))[0])
daisy = str(list(data_dir.glob('daisy/*'))[0])
dandelion = str(list(data_dir.glob('dandelion/*'))[0])
sunflower = str(list(data_dir.glob('sunflowers/*'))[0])

In [ ]:
PIL.Image.open(rose)

In [ ]:
PIL.Image.open(tulip)

In [ ]:
PIL.Image.open(daisy)

In [ ]:
PIL.Image.open(sunflower)

In [ ]:
PIL.Image.open(dandelion)

## Separando los datos

Tensorflow es una de las herramientas que nos ayudará con el todo el trabajo pesado de separación de datos en 3 distintos de subconjuntos: entrenamiento, validación, y prueba. En inglés, los llamaremos *training*, *validation*, y *testing*. También más adelante nos ayudará con la parte de entrenamiento.

Antes de separar los datos debemos definir cuáles son los siguientes parámetros: `batch_size`, `img_height`, `img_width`.



In [ ]:
batch_size =
img_height = 
img_width = 

### Set de entrenamiento

La siguiente celda se encarga de generar, a partir del set de datos completo, el set de datos de entrenamiento.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

### Set de validación

Esta celda, genera el set de datos de validación.

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

### Clases de flores

Tal y como anunciamos anteriormente, Tensorflow conoce las clases que hay en el set de datos. Para ello ejecutamos el código de la siguiente celda.

**IMPORTANTE**: notar que la persona que armó el set de datos en Tensorflow ya había definido la clases en el set de datos, de tal manera que sea posible el entrenamiento con los algoritmos de ML.

In [ ]:
class_names = train_ds.class_names
print(class_names)

También podemos visualizar nuevamente las imágenes con sus respectivas etiquetas utilizando otra herramiento llamada `matplotlib`. Ejecutamos la siguiente celda para ver las fotos de una manera distinta a la anterior, pero esta vez sólo del set de datos de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

## Preprocesando los datos

Para ingresar los datos a la red neuronal es necesario aplicar un tipo de procesamiento previo. Antes, al cargar los set de datos de entrenamien y validación, ya aplicamos algunas transformaciones a los datos. En primer lugar, elegimos qué tamaño tengan las imágenes; porque no todas las imágenes originales tienen las mismas dimensiones. En segundo lugar, exigimos que los datos vengan de a grupos de 32. Dicho en palabras, nuestros datos vendrán en grupos de 32, donde cada canal de la imagen tendrá un ancho por alto de 180 pixels por 180 pixels. Cada canal corresponde al canal rojo, azul y verde de una foto a color.

Al ejecutar la siguiente celda vamos a ver del primer grupo de imágenes para ingresar a la red neuronal.

In [ ]:
for image_batch, labels_batch in train_ds:
  print(f"Image batch size: {image_batch.shape}")
  print(f"Label batch size: {labels_batch.shape}")
  break

### Normalización

Lo próximo que tenemos que hacer es normalizar las imágenes antes de ingresar a la red neuronal. Normalizar es el proceso por el cual los valores de los pixeles de una imagen van a ir entre valores del 0 al 1.

Para demostrar esto, visualizaremos los siguiente. Primero, vamos a mostrar los valores de los pixels de una imagen. Y luego, vamos a ver que el máximo valor de la imagen normalizada es 1 y el mínimo es 0; en el medio pueden haber valores entre el 0 y el 1.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

En este primer caso, visualizamos los valores de cada pixel y vemos que no corresponden entre 0 y 1.

In [ ]:
image_batch, labels_batch = next(iter(train_ds))
image_batch[0]

Aplicamos la normalización a esta misma imagen, y volvemos a mostrar los valores de los pixels.

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
image_batch[0]


Y ahora verificamos que el máximo de la imagen sea 1 y el mínimo 0.

In [ ]:
# Notice the pixel values are now in `[0,1]`.
print(f"Mínimo: {np.min(image_batch[0])}", f"Máximo: {np.max(image_batch[0])}")

## Diseñando la red neuronal

A partir de ahora estamos listos para definir y crear la red neuronal, es decir, explicitar la arquitectura que va a tener nuestra red.

Para ello tenemos la opción de agregar tipos de capas. Recordar que no son todas obligatorias y no hay límite de cuántas agregar (en principio).

Tipos de capas:
* `tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation=None)`
* `tf.keras.layers.MaxPooling2D()`
* `tf.keras.layers.Flatten()`
* `tf.keras.layers.Dropout(rate)`
* `tf.keras.layers.Dense(units, activation=None)`

La idea es que ustedes investiguen y jueguen de tal manera de que se pueda armar una red capaz de poder entrenarse. Es importante empezar de a pasos chiquitos e ir creciendo la red hasta que mejore.


In [ ]:
num_classes = len(class_names)

model = tf.keras.models.Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  # Complete architecture
  tf.keras.layers.Dense(num_classes)
])

Una vez definido la arquitectura, debemos elegir algunos parámetros más. En este caso no es necesario modificar nada, les damos los parámetros de compilación.

In [ ]:
# Choose your learning rate!

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

Con la siguiente celda podemos ver el resúmen de la red neuronal. Finalmente, nos va a decir cuántos parámetros van a ser necesario entrenar con la red que definimos. Tengan cuidado que pueden ser MUCHOS!

In [ ]:
model.summary()

### Tensorboard

Tensorboard nos hará el trabajo más divertido, ya que nos permitirá tener una visualización del trabajo que está realizando la red neuronal, y también, nos dirá cómo le está yendo en términos de desempeño. Ejecutamos la próxima celda para inicializar Tensorboard.

In [ ]:
import os
import datetime

basedir = '/tmp/flowers/'
logdir = os.path.join(basedir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(logdir, exist_ok=True)

%tensorboard --reload_multifile True --logdir {basedir}

## Entrenamiento

Ahora vamos a comenzar a entrenar el algoritmo. Recuerden elegir la cantidad de epochs que deseen entrenar.

In [ ]:
epochs = 0 # Choose your number of epochs. Just try!

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10,
  callbacks=[tensorboard_callback],
)

### Testing

In [ ]:
def test(flower_url, filename):
  path = tf.keras.utils.get_file(filename, origin=flower_url)

  img = tf.keras.utils.load_img(
      path, target_size=(img_height, img_width)
  )
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  print(
      "This image most likely belongs to {} with a {:.2f} percent confidence."
      .format(class_names[np.argmax(score)], 100 * np.max(score))
  )

In [ ]:
test("https://storage.googleapis.com/ds-workshop/flowers/test/roses/3278995478.jpg", "3278995478")

¡Felicitaciones! Entrenaste la red neuronal, pero tengo que mostrate algo, un problema muy común en machine learning.


## Overfitting

El *overfitting* es un problema extremadamente común en los algoritmos de aprendizaje y también es muy intuitivo de cómo aprenden los sistemas en general. Para revisar que nuestra red neuronal comenzó a sufrir de este problema, es necesario graficar la *accuracy* de entrenamiento y la *accuracy* de validación. Ejecutemos la siguiente celda y veamos...

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Lo que estamos viendo es que la accuracy de entrenamiento es muchísimo más alta que la accuracy de validación. Recuerden que la red neuronal se entrena con los datos de entrenamiento, y los datos de validación se utilizan durante el entrenamiento para verificar cómo le está yendo.

Vemos que la red neuronal aprendió "de memoria" los datos de entrenamiento, y cuando le mostramos datos que nunca antes había visto, no supo clasificar bien los datos. Este es el problema de overfitting.

## Data Augmentation

*Data augmentation* es una técnica por la cual podemos ayudar a solucionar el overfitting. La idea es aplicar transformaciones, que tengan sentido, a los datos para generar nuevos datos. Por ejemplo, el caso más común es el de espejar una imagen que puede ayudar a los algortimos a aprender mejor. Nosotros, los humanos, sabemos que una foto de un gato espejada, sigue teniendo un gato y podemos encima decir qué gato es. Entonces, ¿por qué una red neuronal no podría hacer lo mismo?

Veamos cómo Tensorflow nos ayuda a aumentar los datos.

### Transformaciones

Las transformaciones que aplicaremos son:

* Espejar horizontalmente la imagen.
* Rotar de forma aleatoria un factor del número `π`.
* Un zoom aleatorio de un factor dado.

In [ ]:
data_augmentation = tf.keras.models.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

Ahora visualizamos cómo se ve una misma imagen con distintas transformaciones.

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

Ya que sabemos que cómo se ven las transformaciones, vamos a sumarselas al modelo para que pueda se entrenado con ese tipo de datos.

In [ ]:
model = tf.keras.models.Sequential([
  data_augmentation, # We add the augmentation
  tf.keras.layers.Rescaling(1./255),
  # Put the same layers as before.
  tf.keras.layers.Dense(num_classes)
])

In [ ]:
# Again, choose your learning rate.

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

Elegimos nuevamente la cantidad de epochs que queremos entrenar...y esperamos.

In [ ]:
epochs = 0 # Don't forget to set the epochs
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Ejecutamos la celda, y vemos cómo se entreno. O también podemos ir a Tensorboard más arriba a visualizar la accuracy y la loss function.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

¡Hubo una gran mejora en cuanto al overfitting! Ya la accuracies están más cercanas durante todo el entrenamiento.

Ahora probemos si clasifica bien.

In [ ]:
test("https://storage.googleapis.com/ds-workshop/flowers/test/tulips/2208869753.jpg", "2208869753")